## Basic Streams Template
The basic cells needed are included below.  Make any changes and add your sources, analytics and outputs.

For more information refer to: [IBM Streams Python support](https://streamsxtopology.readthedocs.io/en/stable/index.html)

Run the cell below to check which version of the `streamsx` package is installed.  

If you need to upgrade, use

- `import sys`
- `!{sys.executable} -m pip install --user --upgrade streamsx` to upgrade the package.
- Or, use  `!{sys.executable} -m pip install --user streamsx==somever` to install a specific version of the package. 

In [ ]:
import streamsx.topology.context
print("INFO: streamsx package version: " + streamsx.topology.context.__version__)

#For more details uncomment line below.
#!pip show streamsx

In [ ]:
#use the Connect to Instance toolbar button and insert code for config
#e.g.
#from icpd_core import icpd_util
#cfg=icpd_util.get_service_instance_details(name='instance_name')

In [ ]:
#create topology...change the name to something more meaningful
from streamsx.topology.topology import Topology
import streamsx.topology.context

topo = Topology(name="BasicTemplate")

In [ ]:
#add stream...delete and replace this with your streams and analytics
#this is added so the topology will run (needs a source)
#produces a random integer between 1 and 100 every tenth of a second
import random, time
def readings():
    while True:
        time.sleep(0.1)
        yield random.randint(1,100)

source = topo.source(readings)

#create a view to watch the stream data while running
source_view = source.view(name="RandomIntegers", description="Random integers from source stream")

In [ ]:
from streamsx.topology import context

# Disable SSL certificate verification if necessary
cfg[context.ConfigParams.SSL_VERIFY] = False
# submit the topology 'topo'
submission_result = context.submit ("DISTRIBUTED", topo, config = cfg)

# The submission_result object contains information about the running application, or job
if submission_result.job:
    streams_job = submission_result.job
    print ("JobId: ", streams_job.id , "\nJob name: ", streams_job.name)

In [ ]:
#show 30 seconds of data
if submission_result.job:
    queue = source_view.start_data_fetch()
    timeSpan = time.time() + 30;
    while time.time() < timeSpan:
        print(queue.get())
    source_view.stop_data_fetch()


### Cancel Job
The following cell is used to cancel an IBM Streams job. IBM Streams jobs
run forever.  Press the button to cancel to the job.

In [ ]:
submission_result.cancel_job_button()